In [4]:
#Import FindSpark
import findspark
findspark.init('/home/ubuntu/spark-3.3.0-bin-hadoop3')
# Import PySpark
from pyspark.sql import SparkSession
#Start PySpark session
spark = SparkSession.builder.appName('linear_regression').getOrCreate()

In [5]:
# Importing the LinearRegression from Machine Learning library
from pyspark.ml.regression import LinearRegression

In [8]:
# Submitting the traning data
# fromat means that data are in libsvm format inside the sample_linear_regression_data csv file.
traning = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

22/09/09 07:34:24 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [7]:
# Just to view the data
traning.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [9]:
# Creating an instance of our model
# feature col contains the vector - name is features by default
# label is also by default 
# 
lr = LinearRegression(featuresCol='features', labelCol='label', 
                     predictionCol='prediction')

In [11]:
# fitting/traning the model 
# Now we can grab the information concerning this model 
lrModel = lr.fit(traning)

22/09/09 07:37:04 WARN Instrumentation: [69e7c6e2] regParam is zero, which might cause numerical instability and overfitting.


In [13]:
# grabbing the coefficient(współczynnik) of our model
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [15]:
# grabbing the intercept(punkt przecięcia prostej z osią współrzędnych) of our model
lrModel.intercept

0.14228558260358093

In [16]:
# creatning the summary var that holds more of a data
traning_summary = lrModel.summary

In [17]:
# r squared expalains how much variance is expalined by your model
traning_summary.r2

0.027839179518600154

In [29]:
#Get the residuals of the fitted model by type.
#Parameters:	residualsType – The type of residuals which should be returned. Supported options: deviance (default), pearson, working, and response.
traning_summary.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-11.011130022096554|
| 0.9236590911176538|
|-4.5957401897776675|
|  -20.4201774575836|
|-10.339160314788181|
|-5.9552091439610555|
|-10.726906349283922|
|  2.122807193191233|
|  4.077122222293811|
|-17.316168071241652|
| -4.593044343959059|
|  6.380476690746936|
| 11.320566035059846|
|-20.721971774534094|
| -2.736692773777401|
| -16.66886934252847|
|  8.242186378876315|
|-1.3723486332690233|
|-0.7060332131264666|
|-1.1591135969994064|
+-------------------+
only showing top 20 rows



In [30]:
#rootMeanSquaredError (pierwiastek błędu średniokwadratowego) 
traning_summary.rootMeanSquaredError

10.16309157133015

In [20]:
# defining for split
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

22/09/09 07:43:23 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [24]:
# actually splitting the data below
# the list describes how the data is splitted. This time is 70% to 30%
# note - random let's us to avoid splitting the sorted values  
train_data, test_data = all_data.randomSplit([0.7,0.3])

In [22]:
test_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
| -16.26143027545273|(10,[0,1,2,3,4,5,...|
|-15.951512565794573|(10,[0,1,2,3,4,5,...|
| -15.86200932757056|(10,[0,1,2,3,4,5,...|
|-15.780685032623301|(10,[0,1,2,3,4,5,...|
|-15.732088272239245|(10,[0,1,2,3,4,5,...|
|-15.375857723312297|(10,[0,1,2,3,4,5,...|
|-14.822152909751189|(10,[0,1,2,3,4,5,...|
|-14.762758252931127|(10,[0,1,2,3,4,5,...|
|-14.328978509075442|(10,[0,1,2,3,4,5,...|
|-13.867087895158768|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -13.15333560636553|(10,[0,1,2,3,4,5,...|
|-12.977848725392104|(10,[0,1,2,3,4,5,...|
| -12.92222310337042|(10,[0,1,2,3,4,5,...|
| -12.46765638103286|(10,[0,1,2,3,4,5,...|
|-12.198096564661412|(10,[0,1,2,3,4,5,...|
|-12.130353212287929|(10,[0,1,2,3,4,5,...|
+----------

In [9]:
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
|-17.494200356883344|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
|-17.026492264209548|(10,[0,1,2,3,4,5,...|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|
|-16.692207021311106|(10,[0,1,2,3,4,5,...|
| -16.26143027545273|(10,[0,1,2,3,4,5,...|
|-15.780685032623301|(10,[0,1,2,3,4,5,...|
|-15.437384793431217|(10,[0,1,2,3,4,5,...|
|-15.375857723312297|(10,[0,1,2,3,4,5,...|
|-15.359544879832677|(10,[0,1,2,3,4,5,...|
+----------

In [10]:
# Using the describe() method let's us see the difference between dfs that comes from splitted df. 
train_data.describe().show()

+-------+------------------+
|summary|             label|
+-------+------------------+
|  count|               351|
|   mean|0.3381130139537031|
| stddev| 9.854579112377605|
|    min|-23.51088409032297|
|    max| 27.78383192005107|
+-------+------------------+



In [25]:
# to spot the difference we are observing the count parameter of describe() method
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                145|
|   mean|0.36213777180567064|
| stddev| 10.300622365418276|
|    min|-26.805483428483072|
|    max| 22.923352376063196|
+-------+-------------------+



In [26]:
# Fits a model to the input dataset with optional parameters.
correct_model = lr.fit(train_data)

22/09/09 07:50:58 WARN Instrumentation: [824d7995] regParam is zero, which might cause numerical instability and overfitting.


In [27]:
# Evaluates the model on a test dataset.
# Parameters:	dataset – Test dataset to evaluate model on, where dataset is an instance of pyspark.sql.DataFrame
test_results = correct_model.evaluate(test_data)

In [14]:
test_results.rootMeanSquaredError

11.401233691872427

In [32]:
# Usually we are test data that has no label. Below we have create the var that does not containt the label value to mimic
# the possible scenario
unlabeled_data = test_data.select('features')

In [33]:
# No label column
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [34]:
# Transforms the input dataset with optional parameters.

# Parameters:	
# dataset – input dataset, which is an instance of pyspark.sql.DataFrame
# params – an optional param map that overrides embedded params.
# Returns:	
# transformed dataset
predictions = correct_model.transform(unlabeled_data)

In [35]:
# We now have the our predictions column
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|  1.7204627307648381|
|(10,[0,1,2,3,4,5,...|  1.3474172366092387|
|(10,[0,1,2,3,4,5,...|   1.293137399625605|
|(10,[0,1,2,3,4,5,...|  0.3701587286554809|
|(10,[0,1,2,3,4,5,...|  0.5147031115531919|
|(10,[0,1,2,3,4,5,...|  1.9962327386125114|
|(10,[0,1,2,3,4,5,...| 0.14541311779189509|
|(10,[0,1,2,3,4,5,...| -1.6799631861397402|
|(10,[0,1,2,3,4,5,...| -1.2743257653796507|
|(10,[0,1,2,3,4,5,...|  1.5168636942153444|
|(10,[0,1,2,3,4,5,...| -1.1043175009548138|
|(10,[0,1,2,3,4,5,...|-0.19911013241290848|
|(10,[0,1,2,3,4,5,...|   2.563025167871099|
|(10,[0,1,2,3,4,5,...|  -1.818867159460676|
|(10,[0,1,2,3,4,5,...|  1.2964604368871206|
|(10,[0,1,2,3,4,5,...|  1.3187623910409172|
|(10,[0,1,2,3,4,5,...|  0.8119666160218004|
|(10,[0,1,2,3,4,5,...|-0.00275614191803...|
|(10,[0,1,2,3,4,5,...|  -2.769939906115894|
|(10,[0,1,2,3,4,5,...| 0.0733615